In [2]:
import cvxpy as cp
import numpy as np

from basis_generator import basis_gs, sel_seq

# SZDM N=6 Input parameters

In [5]:
dimX = 3
dim_base = 2
num_obs = 6
len_seq = 3
num_out = 2
remove_last_out = False

SZDM6_events = [((1, 0, 0), (0, 2, 3)),
            ((1, 0, 0), (1, 3, 5)),
            ((0, 1, 0), (1, 2, 3)),
            ((0, 0, 1), (0, 3, 4)),
            ((0, 0, 1), (3, 4, 5))]

In [ ]:
X_basis = basis_gs(dim = dim_base,
            num_obs = num_obs,
            len_seq = len_seq,
            num_out = num_out,
            level = 1,
            remove_last_out = remove_last_out,
            stop = 1000)

(1728, 1728)


In [ ]:
sequences = sel_seq(num_obs = num_obs,
                len_seq = len_seq,
                num_out = num_out,
                remove_last_out = False)

C = np.zeros(X_basis[0].shape)
for i, seq_row in enumerate(sequences):
    if seq_row in SZDM6_events:
        C[i,i] = 1

# Optimization with the First Level of the Hierarchy

In [ ]:
alpha = cp.Variable((len(X_basis), 1))
X = cp.Variable(X_basis[0].shape, symmetric=True)

constraints = [X >> 0]
constraints += [
    X == sum([alpha[i]*X_basis[i] for i in range(len(X_basis))])
]
constraints += [
    cp.trace(X) == num_obs**len_seq
]

In [ ]:
prob = cp.Problem(cp.Maximize(cp.trace(C@X)), constraints)

In [ ]:
prob.solve(solver=cp.MOSEK, verbose=True)

print(prob.status)
print("The optimal value is", prob.value)